Agenda

1. Window Func
2. Jobs and Pipelines - Demo

In [0]:
id name  email  insertdate
1  aa     abc.com   29/10/2025



1  aa   cdc.com  ===> csv fileName_30102025.csv




In [0]:
f_df  = spark.read.option("header",True)\
                    .option("inferSchema",True)\
                    .csv("/Volumes/sample_cat/default/volume_csv/sample_dir/atc_pre_departure_delays_30102025.csv")

In [0]:
from pyspark.sql.functions import *

In [0]:
f_df = f_df.withColumn("raw_date", regexp_extract("filename", r"(\d{8})", 1))

# Convert from ddMMyyyy → yyyy-MM-dd
f_df = f_df.withColumn("file_date", to_date(f_df["raw_date"], "ddMMyyyy"))

In [0]:
f_df.withColumn("filename",f_df["_metadata.file_path"])\
    .withColumn("raw_date", regexp_extract("filename", r"(\d{8})", 1))\
    .withColumn("Insert_date", to_date(col("raw_date"), "ddMMyyyy"))\
    .withColumn("Updated_date",col("Insert_date")).display() 

In [0]:
f_df.select("APT_ICAO","FLT_DATE").display()

row_number()
EBAW	2016-01-01   1
EBAW	2016-01-01   2


dense_rank()
EBAW	2016-01-01   1
EBAW	2016-01-01   1






In [0]:
from pyspark.sql.window import Window

In [0]:
win = Window.partitionBy("APT_ICAO").orderBy(col("FLT_DATE").desc())

In [0]:
f_df.withColumn("rn",row_number().over(win))\
    .filter(col('rn') == 1).limit(3).display()

YEAR,MONTH_NUM,MONTH_MON,FLT_DATE,APT_ICAO,APT_NAME,STATE_NAME,FLT_DEP_1,FLT_DEP_IFR_2,DLY_ATC_PRE_2,FLT_DEP_3,DLY_ATC_PRE_3,rn
2016,12,DEC,2016-12-31,EBAW,Antwerp,Belgium,4,null,null,1,0,1
2016,12,DEC,2016-12-31,EBBR,Brussels,Belgium,167,167,538,132,32,1
2016,12,DEC,2016-12-31,EBCI,Charleroi,Belgium,48,48,0,44,0,1


In [0]:
f_df.withColumn("rn",rank().over(win))\
    .filter(col('rn') == 1).limit(3).display()

YEAR,MONTH_NUM,MONTH_MON,FLT_DATE,APT_ICAO,APT_NAME,STATE_NAME,FLT_DEP_1,FLT_DEP_IFR_2,DLY_ATC_PRE_2,FLT_DEP_3,DLY_ATC_PRE_3,rn
2016,12,DEC,2016-12-31,EBAW,Antwerp,Belgium,4,null,null,1,0,1
2016,12,DEC,2016-12-31,EBBR,Brussels,Belgium,167,167,538,132,32,1
2016,12,DEC,2016-12-31,EBCI,Charleroi,Belgium,48,48,0,44,0,1


In [0]:
f_df.withColumn("rn",dense_rank().over(win))\
    .filter(col('rn') == 1).limit(3).display()

YEAR,MONTH_NUM,MONTH_MON,FLT_DATE,APT_ICAO,APT_NAME,STATE_NAME,FLT_DEP_1,FLT_DEP_IFR_2,DLY_ATC_PRE_2,FLT_DEP_3,DLY_ATC_PRE_3,rn
2016,12,DEC,2016-12-31,EBAW,Antwerp,Belgium,4,null,null,1,0,1
2016,12,DEC,2016-12-31,EBBR,Brussels,Belgium,167,167,538,132,32,1
2016,12,DEC,2016-12-31,EBCI,Charleroi,Belgium,48,48,0,44,0,1
